# Weather Prediction
By Yash Patel, Vinod Raman, Seamus Somerstep, and Unique Subedi

## Executive Summary

## Introduction

## Data Sources and Preparation

## Evaluation Pipeline

## Prediction Models

### Baselines

#### Previous Day Predictor

#### Historical Average Predictor

#### Weighted Average Predictor

### Time Series Models

### Regression-Based Models

#### Ordinary Least Squares Regression (OLS)

#### Lasso Regression 

#### Ridge Regression

#### Multilayer Perceptron (MLP)

### Ensemble Methods

#### Gradient Boosted Trees

#### Random Forest

## Model Selection and Evaluation

## Conclusion